<a href="https://colab.research.google.com/github/DileepNalle78/pyspark__DileepNalle/blob/main/adv_scala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!wget -q https://github.com/scala/scala/releases/download/v2.12.18/scala-2.12.18.deb
!tar xf spark-3.4.1-bin-hadoop3.tgz
!dpkg -i scala-2.12.18.deb

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

!curl -Lo coursier https://git.io/coursier-cli && chmod +x coursier
!./coursier launch --fork almond --scala 2.12.10 -- --install

(Reading database ... 130340 files and directories currently installed.)
Preparing to unpack scala-2.12.18.deb ...
Unpacking scala (2.12.18-400) over (2.12.18-400) ...
Setting up scala (2.12.18-400) ...
Processing triggers for man-db (2.10.2-1) ...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42577  100 42577    0     0  99924      0 --:--:-- --:--:-- --:--:-- 99924
Error: /root/.local/share/jupyter/kernels/scala already exists, pass --force to force erasing it


In [53]:
!ls /usr/lib/jvm/java-11-openjdk-amd64
!ls /content/spark-3.4.1-bin-hadoop3
!ls /content/scala-2.12.18.deb
!echo $JAVA_HOME
!echo $SPARK_HOME

bin  conf  docs  include  jmods  legal	lib  man  release
bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin
/content/scala-2.12.18.deb
/usr/lib/jvm/java-11-openjdk-amd64
/content/spark-3.4.1-bin-hadoop3


In [54]:
!pip install -q findspark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Colab Scala Spark") \
    .getOrCreate()

spark

In [66]:
%%writefile HelloWorld_hi.scala
import org.apache.spark.sql.SparkSession

object HelloWorld_hi {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder()
      .appName("ScalaAdvancedTransformations")
      .master("local[*]")
      .config("spark.driver.memory", "1g")
      .getOrCreate()

    println("Spark session created successfully.")
    spark.stop()
  }
}

Overwriting HelloWorld_hi.scala


In [56]:
!scalac HelloWorld_hi.scala

HelloWorld_hi.scala:1: error: object apache is not a member of package org
import org.apache.spark.sql.SparkSession
           ^
HelloWorld_hi.scala:5: error: not found: value SparkSession
    val spark = SparkSession.builder()
                ^
two errors found


In [57]:
!scala HelloWorld_hi

No such file or class on classpath: HelloWorld_hi


In [67]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld_hi.scala

In [68]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld_hi

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 06:39:30 INFO SparkContext: Running Spark version 3.4.1
25/08/01 06:39:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 06:39:31 INFO ResourceUtils: ==============================================================
25/08/01 06:39:31 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 06:39:31 INFO ResourceUtils: ==============================================================
25/08/01 06:39:31 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 06:39:31 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 06:39

In [60]:
%%writefile HelloWorld.scala
import org.apache.spark.sql.SparkSession
object HelloWorld {
    def main(args: Array[String]): Unit = {
        val spark = SparkSession.builder()
          .appName("ScalaAdvancedTransformations")
          .master("local[*]")
          .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
          .getOrCreate()
        val sc = spark.sparkContext

        ///////////////////////////////////////////create an accumulator variable
        val errorCount = sc.longAccumulator("Error Count")

        // Sample rdd with some log line
        val data = sc.parallelize(Seq("INFO Start", "ERROR Failure1", "ERROR Failure2", "INFO End"))

        //Use the accumulator
        data.foreach(line => {
          if (line.contains("ERROR")) {
            errorCount.add(1)
          }
        })

        println("Total errors: " + errorCount.value)
       spark.stop()
    }
}

Overwriting HelloWorld.scala


In [62]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld.scala

In [63]:

!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 06:37:07 INFO SparkContext: Running Spark version 3.4.1
25/08/01 06:37:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 06:37:07 INFO ResourceUtils: ==============================================================
25/08/01 06:37:07 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 06:37:07 INFO ResourceUtils: ==============================================================
25/08/01 06:37:07 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 06:37:07 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 06:37

In [69]:
%%writefile broadcast.scala
import org.apache.spark.sql.SparkSession
object broadcast {
    def main(args: Array[String]): Unit = {
        val spark = SparkSession.builder()
          .appName("ScalaAdvancedTransformations")
          .master("local[*]")
          .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
          .getOrCreate()
          val sc = spark.sparkContext
          // Create a lookup map to broadcast
          val countryCodes = Map("USA" -> "US", "India" -> "IN", "China" -> "CN")
          // Create a broadcast variable
          val countryCodesBroadcast = sc.broadcast(countryCodes)
          // Sample RDD with country names
          val countries = sc.parallelize(Seq("USA", "India", "China", "Japan"))
          // Map using the broadcast variable
          val countryCodesMapped = countries.map(country => (country, countryCodesBroadcast.value.getOrElse(country, "Unknown")))
          // Collect and print the result
          countryCodesMapped.collect().foreach(println)
       spark.stop()
    }
}

Writing broadcast.scala


In [71]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" broadcast.scala

In [72]:

!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" broadcast

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 06:54:15 INFO SparkContext: Running Spark version 3.4.1
25/08/01 06:54:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 06:54:15 INFO ResourceUtils: ==============================================================
25/08/01 06:54:15 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 06:54:15 INFO ResourceUtils: ==============================================================
25/08/01 06:54:15 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 06:54:15 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 06:54

In [75]:
%%writefile data.scala
import org.apache.spark.sql.SparkSession
object data {
    def main(args: Array[String]): Unit = {
        val spark = SparkSession.builder()
          .appName("ScalaAdvancedTransformations")
          .master("local[*]")
          .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
          .getOrCreate()
        val sc = spark.sparkContext
        val data = sc.parallelize(1 to 1000000) // Removed underscore

        val evenNumbers = data.filter(x => x % 2 == 0).cache() // Fixed syntax

        println("count: " + evenNumbers.count())

        println("sum: " + evenNumbers.sum())
        spark.stop()
    }
}

Overwriting data.scala


In [76]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" data.scala

In [77]:

!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" data

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 07:01:42 INFO SparkContext: Running Spark version 3.4.1
25/08/01 07:01:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 07:01:42 INFO ResourceUtils: ==============================================================
25/08/01 07:01:42 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 07:01:42 INFO ResourceUtils: ==============================================================
25/08/01 07:01:42 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 07:01:42 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 07:01

In [83]:
%%writefile memory.scala
import org.apache.spark.sql.SparkSession
import org.apache.spark.storage.StorageLevel // Import StorageLevel

object memory {
    def main(args: Array[String]): Unit = {
        val spark = SparkSession.builder()
          .appName("ScalaAdvancedTransformations")
          .master("local[*]")
          .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
          .getOrCreate()
        val sc = spark.sparkContext

        val bigData = sc.parallelize(1 to 1000000)

        val squares = bigData.map(x => x * x).persist(StorageLevel.MEMORY_AND_DISK) // Corrected StorageLevel

        println("First 5: " + squares.take(5).mkString(", "))

        println("Count: " + squares.count())


        spark.stop()
    }
}

Overwriting memory.scala


In [84]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" memory.scala

In [85]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" memory

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 07:10:57 INFO SparkContext: Running Spark version 3.4.1
25/08/01 07:10:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 07:10:58 INFO ResourceUtils: ==============================================================
25/08/01 07:10:58 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 07:10:58 INFO ResourceUtils: ==============================================================
25/08/01 07:10:58 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 07:10:58 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 07:10